In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    max_retries=2,
)

In [8]:
map_template = "Write a concise summary of the following: {docs}."
map_prompt = ChatPromptTemplate([("human", map_template)])
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [9]:
# Reduce
reduce_template = """
    The following is a set of summaries:
    {docs}
    Take these and distill it into a final, consolidated summary
    of the main themes.
"""
reduce_prompt = ChatPromptTemplate([("human", reduce_template)])
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [10]:
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)
# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=1000,
)

In [11]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=True,
)

In [13]:
documents = TextLoader('./documents/text2.txt', encoding='utf8')

docs = documents.load()

print(len(docs))

1


In [31]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0,
)

split_docs = text_splitter.split_documents(docs)
print(f"Generated {len(split_docs)} documents.")

Created a chunk of size 1296, which is longer than the specified 1000
Created a chunk of size 1040, which is longer than the specified 1000


Generated 13 documents.


In [43]:
print(f'Number of tokens as per the given model: - {[llm.get_num_tokens(text = i.page_content) for i in split_docs]}')
print(f'Number of characters inside each chunk: - {[len(i.page_content) for i in split_docs]}')

Number of tokens as per the given model: - [981, 965, 985, 968, 1011, 976, 912, 1278, 996, 215, 1022, 1018, 313]
Number of characters inside each chunk: - [3931, 3544, 4222, 4310, 4036, 4180, 3936, 3599, 3525, 1024, 4050, 3320, 823]


In [33]:
result = map_reduce_chain.invoke(split_docs)

e:\Codes\Data Sciene\AI\Long-Text-Document-Summarization\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
e:\Codes\Data Sciene\AI\Long-Text-Document-Summarization\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (4477 > 1024). Running thi

In [34]:
result

{'input_documents': [Document(metadata={'source': './documents/text2.txt'}, page_content='Table of Contents\n\n  * Agent System Overview\n  * Component One: Planning\n    * Task Decomposition\n    * Self-Reflection\n  * Component Two: Memory\n    * Types of Memory\n    * Maximum Inner Product Search (MIPS)\n  * Component Three: Tool Use\n  * Case Studies\n    * Scientific Discovery Agent\n    * Generative Agents Simulation\n    * Proof-of-Concept Examples\n  * Challenges\n  * Citation\n  * References \n\nBuilding agents with LLM (large language model) as its core controller is a\ncool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer\nand BabyAGI, serve as inspiring examples. The potentiality of LLM extends\nbeyond generating well-written copies, stories, essays and programs; it can be\nframed as a powerful general problem solver.\n\n# Agent System Overview#\n\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain,\ncomplemented by several k

In [36]:
for i in result['intermediate_steps']:
    print(i)
    print("*"*20)

A Large Language Model (LLM) can be used as the core controller in an autonomous agent system. An overview of this system consists of three key components:

1. **Planning**: Breaking down complex tasks into smaller, manageable subgoals through task decomposition and self-reflection. This involves techniques like Chain of Thought (CoT) and Tree of Thoughts (Yao et al. 2023) to decompose tasks into smaller steps.
2. **Memory**: Utilizing both short-term and long-term memory to retain and recall information over extended periods. This can be achieved through in-context learning and external vector stores for fast retrieval.
3. **Tool Use**: Learning to call external APIs for additional information, such as current data, code execution, and access to proprietary sources.

The system uses a combination of prompting techniques, task-specific instructions, and human inputs to achieve task decomposition. Additionally, an external classical planner can be used through the LLM+P approach (Liu et

In [39]:
print(result['output_text'])

**Enhancing Large Language Models (LLMs) for Autonomous Decision-Making and Task Completion**

**Main Themes:**

1. **Tool Use and External Expertise**: Incorporating external tools, APIs, and expert modules to extend LLM capabilities and access additional information.
2. **Memory Types and Self-Reflection**: Utilizing short-term and long-term memory through in-context learning and external vector stores, as well as implementing frameworks like ReAct, Reflexion, and Chain of Hindsight (CoH) for self-reflection and iterative improvement.
3. **Planning and Task Decomposition**: Employing techniques like Chain of Thought (CoT) and Tree of Thoughts for task decomposition and self-reflection, as well as using languages like Planning Domain Definition Language (PDDL) for defining planning domains.
4. **Optimization and Retrieval**: Utilizing algorithms like MIPS, LSH, ANNOY, HNSW, FAISS, and ScaNN to optimize retrieval speed and improve text generation.

**Key Applications and Challenges**

